In [4]:
using PauliPropagation
using Plots
using Printf
using LinearAlgebra
using LaTeXStrings
using ProgressMeter

In [5]:
function thermalStateExpectation(circuit, nl, nq, operator; max_weight=nq, min_abs_coeff=0)
    pstr = PauliString(nq, :I, 1)
    psum = PauliSum(pstr)

    expectations = Float64[]

    @showprogress for layers in 1:nl
        psum = propagate!(circuit, psum; max_weight, min_abs_coeff, normalization=true)
        layers += 1
        expectation = getcoeff(psum, operator)
        expectations = push!(expectations, expectation)
    end
    println(length(psum))
    return expectations
end

thermalStateExpectation (generic function with 1 method)

In [6]:
theta = 0.01im
θ = theta
nq = 30

nl = 1

pstr = PauliString(nq, :I, 1)

# the computational basis matrices
rzmat = tomatrix(PauliRotation(:Z, 1), -θ)
rxmat = tomatrix(PauliRotation(:X, 1), -θ)
rzzmat = tomatrix(PauliRotation([:X, :X], [1, 2]), -θ)

# the Pauli basis matrices
rzptm = calculateptm(rzmat; heisenberg=false)
rxptm = calculateptm(rxmat; heisenberg=false)
rzzptm = calculateptm(rzzmat; heisenberg=false)

# the Pauli basis transfermaps
rzmap = totransfermap(rzptm)
rxmap = totransfermap(rxptm)
rzzmap = totransfermap(rzzptm);
topology = bricklayertopology(nq)

# the circuit
circuit = Gate[]
for i in 1:nl
    #append!(circuit, TransferMapGate(rxmap, ii) for ii in 1:nq);
    append!(circuit, TransferMapGate(rzmat, ii) for ii in 1:nq);
    append!(circuit, TransferMapGate(rzzmat,collect(pair)) for pair in topology);
end

In [8]:
obs_i = 20
obs_j = 21
observable = PauliString(nq, [:Z,:Z], [obs_i,obs_j])

nl = 20
trunc_coeffs = 2.0 .^ (-14:-1:-18)

time = range(0.01, 0.01*nl, nl)

plot(ylabel=L"Absolute error of $\langle Z_{20}Z_{21} \rangle$", xlabel=L"\beta", title = L"$H = \sum_{i=1} -Z_i - X_iX_{i+1}$")

for j in trunc_coeffs
    expectation = thermalStateExpectation(circuit, nl, nq, observable; min_abs_coeff=j, max_weight = 9)
    plot!(time, expectation, label = L"$2^{%$(round(log2(j), digits=2))}$", legend=:topleft, lw=2, markersize=4, marker=:circle, alpha=0.5)
end


display(plot!())


Progress: 100%|█████████████████████████████████████████| Time: 0:00:12


31030

Progress:  15%|███████                                  |  ETA: 0:00:01

Progress:  90%|█████████████████████████████████████    |  ETA: 0:00:03